In [1]:
import lightgbm as lgb
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy import sparse 
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import chi2, SelectPercentile,SelectKBest
import gc

In [2]:
from scipy.sparse import csr_matrix
def save_csr(matrix,file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    np.save(data_file,matrix.data)
    print('data save')
    np.save(indices_file,matrix.indices)
    print('indices save')
    np.save(indptr_file,matrix.indptr)
    print('indptr save')
    print('save done!')
    
def load_csr(file_prefix='csr'):
    data_file = file_prefix + '_data.npy'
    indices_file = file_prefix + '_indices.npy'
    indptr_file = file_prefix + '_indptr.npy'
    data = np.load(data_file)
    print('data load')
    indices = np.load(indices_file)
    print('indices load')
    indptr = np.load(indptr_file)
    print('indptr load')
    csr = csr_matrix((data, indices, indptr))
    return csr

In [ ]:
first = 2500,2400,2300,2700,3000,3500

In [3]:
first = 2400
train_csr = load_csr('../../data/features/xgb/train_csr_{}'.format(first))
predict_csr = load_csr('../../data/features/xgb/predict_csr_{}'.format(first))
train_y = np.load('../../data/features/xgb/train_y_f620_b70_col17k.npy')
predicted_file = pd.read_csv('../../data/features/xgb/predict_f620_b70_col17k.csv')
print(train_csr.shape)

data load
indices load
indptr load
data load
indices load
indptr load
(2992639, 2257)


In [ ]:
# slot_mean
# fchannel
# appid
# model

In [3]:
train_csr = load_csr('../../data/features/xgb/train_csr_2500')
predict_csr = load_csr('../../data/features/xgb/predict_csr_2500')
train_instance = pd.read_pickle('../../data/features/xgb/train_noclean_bytime.pkl')[['instance_id']].copy()
train_y = np.load('../../data/features/xgb/train_y_f620_b70_col17k.npy')
predicted_file = pd.read_csv('../../data/features/xgb/predict_f620_b70_col17k.csv')
test_instance = predicted_file[['instance_id']].copy()
all_instance = pd.concat([train_instance,test_instance],sort=False)

X_u = pd.read_pickle('../../data/features/agg/svd_utag_slotId_appearance.pkl')

X_u = all_instance.merge(X_u,on='instance_id',how='inner')
X_u.drop('instance_id',inplace=True,axis=1)
print(X_u.shape)

cut_bin = 30
# selected_col = [col for col in X_u.columns if 'utagsIndividual_svd' not in col]
# X_u = X_u[selected_col]
for col in tqdm(X_u.columns):
        X_u[col] = pd.cut(X_u[col], bins=cut_bin, labels=False, retbins=True, right=False)[0]  
        
print(train_csr.shape)
train_csr = sparse.hstack(
    (sparse.csr_matrix(X_u.iloc[:train_csr.shape[0]]), train_csr), 'csr').astype(
    'float32')
predict_csr = sparse.hstack(
    (sparse.csr_matrix(X_u.iloc[train_csr.shape[0]:]), predict_csr), 'csr').astype('float32')
print(train_csr.shape)
print(predict_csr.shape)

data load
indices load
indptr load
data load
indices load
indptr load
(2992639, 1)


100%|██████████| 32/32 [00:06<00:00,  4.83it/s]


(2992639, 2363)


KeyboardInterrupt: 

In [6]:
sparse.csr_matrix(X_u.iloc[train_csr.shape[0]:]).shape

(0, 32)

# define lgb

In [15]:
lgb_model = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=32, reg_alpha=0, reg_lambda=0.1,
    max_depth=-1, n_estimators=5000, objective='binary',
    subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.05, random_state=1001, n_jobs=16
)


In [ ]:
predict_result = predicted_file[['instance_id']].copy()
predict_result['predicted_score'] = 0

n_fold = 5
seed=np.random.randint(1000)
gc.collect()
skf = StratifiedKFold(n_splits=n_fold, random_state=seed, shuffle=True)
best_score = []
oof = np.zeros(len(train_y))
for index, (train_index, test_index) in enumerate(skf.split(train_csr, train_y)):
    lgb_model.fit(train_csr[train_index], train_y[train_index],
                      eval_set=[(train_csr[train_index], train_y[train_index]),
                                (train_csr[test_index], train_y[test_index])], early_stopping_rounds=200,verbose=200)
    best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
    print(best_score)
    print('cv:',np.mean(best_score))
    test_pred = lgb_model.predict_proba(predict_csr, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred
    oof[test_index] = lgb_model.predict_proba(train_csr[test_index], num_iteration=lgb_model.best_iteration_)[:, 1]
    gc.collect()

predict_result['predicted_score'] = predict_result['predicted_score']/n_fold

Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_logloss: 0.415636	valid_1's binary_logloss: 0.416723
[400]	valid_0's binary_logloss: 0.413396	valid_1's binary_logloss: 0.415585
[600]	valid_0's binary_logloss: 0.411779	valid_1's binary_logloss: 0.41521
[800]	valid_0's binary_logloss: 0.410322	valid_1's binary_logloss: 0.415023
[1000]	valid_0's binary_logloss: 0.408952	valid_1's binary_logloss: 0.414914
[1200]	valid_0's binary_logloss: 0.407632	valid_1's binary_logloss: 0.41485
[1400]	valid_0's binary_logloss: 0.406369	valid_1's binary_logloss: 0.414807
[1600]	valid_0's binary_logloss: 0.405143	valid_1's binary_logloss: 0.414753
[1800]	valid_0's binary_logloss: 0.403957	valid_1's binary_logloss: 0.414714
[2000]	valid_0's binary_logloss: 0.402794	valid_1's binary_logloss: 0.414682
[2200]	valid_0's binary_logloss: 0.401645	valid_1's binary_logloss: 0.414674
Early stopping, best iteration is:
[2054]	valid_0's binary_logloss: 0.402478	valid_1's binary_lo

In [ ]:
colnum = train_csr.shape[1]
cv_loss = np.mean(best_score)
now = str(np.random.randint(1000000))
predict_result[['instance_id', 'predicted_score']].to_csv( "submission/%s_lgbUtagSlotApran_n%d_b%d_1h_col%d.csv" % (now, n_fold, 70, colnum), index=False)
np.save('submission/oof_%s_%.5f'%(now, cv_loss), oof) 